In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## အဆင့် ၁: ဖွဲ့စည်းထားသော ထုတ်လွှင့်ချက်များအတွက် Pydantic မော်ဒယ်များ သတ်မှတ်ပါ

ဤမော်ဒယ်များသည် လူကြိုက်များသော **schema** ကို သတ်မှတ်ပေးသည်။ Pydantic နှင့်အတူ `response_format` ကို အသုံးပြုခြင်းဖြင့် အောက်ပါအချက်များကို သေချာစေသည်-
- ✅ အမျိုးအစား-လုံခြုံသော ဒေတာထုတ်ယူခြင်း
- ✅ အလိုအလျောက် စစ်ဆေးခြင်း
- ✅ အခမဲ့စာသားတုံ့ပြန်ချက်များမှ ပေးပို့မှုအမှားများ မရှိစေရန်
- ✅ အကွက်များအပေါ် မူတည်၍ အတိုက်အခံ လမ်းကြောင်းခွဲခြားမှု အလွယ်တကူလုပ်ဆောင်နိုင်ခြင်း


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## အဆင့် ၂: ဟိုတယ်Booking ကိရိယာကို ဖန်တီးခြင်း

ဒီကိရိယာကို **availability_agent** က အခန်းတွေအတွက် ရရှိနိုင်စွမ်းရှိမရှိ စစ်ဆေးဖို့ခေါ်မယ်။ `@ai_function` decorator ကို အသုံးပြုပြီး-
- Python function ကို AI ခေါ်နိုင်တဲ့ ကိရိယာတစ်ခုအဖြစ်ပြောင်းလဲခြင်း
- LLM အတွက် JSON schema ကို အလိုအလျောက်ဖန်တီးခြင်း
- ပုံစံမှန်ကန်မှု စစ်ဆေးခြင်း
- ကိုယ်စားလှယ်တွေက အလိုအလျောက်ခေါ်နိုင်စေရန်

ဒီဒါမိုအတွက်-
- **Stockholm, Seattle, Tokyo, London, Amsterdam** → အခန်းရှိ ✅
- **အခြားမြို့တွေ** → အခန်းမရှိ ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## အဆင့် ၃: လမ်းညွှန်မှုအတွက် အခြေအနေ ဖန်တီးမှု မှတ်သားချက်များကို သတ်မှတ်ပါ

ဤ ရုပ်သန်းများသည် ကိုယ်စားလှယ်၏ တုံ့ပြန်ချက်ကို စစ်ဆေးပြီး workflow တွင် မည်သည့်လမ်းကြောင်းကို သွားရမည်ကို ဆုံးဖြတ်သည်။

**အဓိကပုံစံ:**
1. စာတန်းကို `AgentExecutorResponse` ဖြစ်မရှိ စစ်ဆေးပါ
2. ဖွဲ့စည်းတည်ဆောက်ထားသော output ကို (Pydantic မော်ဒယ်) ကောက်နုတ်ပါ
3. လမ်းညွှန်မှု ထိန်းချုပ်ရန် `True` သို့မဟုတ် `False` ကို ပြန်လည်ပေးပါ

workflow သည် ဒီအခြေအနေများကို **အနားပိုင်းများ**တွင် မည်သည့် executor ကို နောက်တစ်ဆင့်ခေါ်မည်ကို ဆုံးဖြတ်ရန် သုံးမည်ဖြစ်သည်။


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Step 4: စိတ်ကြိုက်ပြသသည့် Executor ဖန်တီးခြင်း

Executors သည် transformation များ သို့မဟုတ် side effect များကို ဆောင်ရွက်သော workflow အစိတ်အပိုင်းများဖြစ်သည်။ နောက်ဆုံးရလဒ်ကို ပြသနိုင်ရန် စိတ်ကြိုက် executor ကို `@executor` decorator ဖြင့် ဖန်တီးပါသည်။

**အဓိက အယူအဆများ:**
- `@executor(id="...")` - function ကို workflow executor အဖြစ် စာရင်းသွင်းသည်
- `WorkflowContext[Never, str]` - input/output အတွက် type hints
- `ctx.yield_output(...)` - နောက်ဆုံး workflow ရလဒ်ကို yield ပြုလုပ်သည်


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## ခြေလှမ်း ၅: ပတ်ဝန်းကျင်အချက်အလက်များကို ပွတ်နှံရန်

LLM client ကို ပြင်ဆင်ပါ။ ဤနမူနာအတွက် အလုပ်လုပ်သော အခြေအနေများမှာ-
- **GitHub Models** (GitHub token နှင့် အခမဲ့ အဆင့်)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## အဆင့် ၆: စနစ်တကျ အထွက်များနှင့် AI ကိုယ်စားလှယ်များ ဖန်တီးခြင်း

**အထူးပြုထားသော ကိုယ်စားလှယ်သုံးယောက်**ကို `AgentExecutor` နဲ့ ထောက်ပံ့ပြီး ဖန်တီးပါသည်-

၁။ **availability_agent** - စက်ပစ္စည်းကို အသုံးပြုပြီး ဟိုတယ်ရရှိနိုင်မှုကို စစ်ဆေးသည်  
၂။ **alternative_agent** - အခန်းမရှိပါက အစားထိုးမြို့များကို အကြံပြုသည်  
၃။ **booking_agent** - အခန်းရရှိနိုင်သည်ဆိုပါက မှာကြားရန် ညွှန်ကြားသည်  

**အဓိက အင်္ဂါရပ်များ-**  
- `tools=[hotel_booking]` - ကိုယ်စားလှယ်အတွက် စက်ပစ္စည်းကို ထောက်ပံ့သည်  
- `response_format=PydanticModel` - ဖွဲ့စည်းထားသော JSON အထွက်ကို အတင်းအကျပ် ပြုလုပ်သည်  
- `AgentExecutor(..., id="...")` - လုပ်ငန်းစဉ်အသုံးပြုမှုအတွက် ကိုယ်စားလှယ်ကို လှမ်းတင်သည်


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## ခြေလှမ်း ၇: ရှေ့ပြေးအခြေအနေ ပါဝင်သော များဖြင့် Workflow ကို တည်ဆောက်ခြင်း

ယခု `WorkflowBuilder` ကို အသုံးပြု၍ ရှေ့ပြေးအခြေအနေဖြင့် လမ်းကြောင်းများပါသော ဂရပ်ကို တည်ဆောက်မည်။

**Workflow ဖွဲ့စည်းပုံ:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**အဓိက Method များ:**
- `.set_start_executor(...)` - ဝင်ပေါက်နေရာကို သတ်မှတ်သည်
- `.add_edge(from, to, condition=...)` - ရှေ့ပြေးအခြေအနေပါသော လမ်းကြောင်းကို ထည့်သွင်းသည်
- `.build()` - Workflow ကို နောက်ဆုံးဖြစ်အောင် ပြုလုပ်သည်


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Step 8: Run Test Case 1 - City WITHOUT Availability (Paris)

Paris တွင် အခန်းမရှိခြင်းလမ်းကြောင်းကို စမ်းသပ်ကြည့်ရအောင် (ကျွန်ုပ်တို့၏ အတုအယောင်တွင် အခန်းမရှိပါ)။


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## အဆင့် ၉: စမ်းသပ်မှုကိစ္စ ၂ ကို လည်ပတ်ပါ - ရှိနိုင်မှုဝင်သော မြို့တော် (စတောက်ဟိုမ်)

ယခုအခါ ကျွန်ုပ်တို့၏ စမ်းသပ်ချက်တွင်ရှိသော အခန်းများပါသော စတောက်ဟိုမ်မြို့တွင် ဟိုတယ်များကို တောင်းဆိုပြီး **ရှိနိုင်မှု** လမ်းကြောင်းကို စမ်းသပ်ကြည့်ပါမည်။


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## အချက်အလက်အဓိကယူဆချက်များနှင့် နောက်တစ်ဆင့်များ

### ✅ သင်သင်ထားသောအရာများ

1. **WorkflowBuilder ပုံစံ**
   - `.set_start_executor()` ကို အသုံးပြု၍ ထွက်ပေါက်အချက်ကို သတ်မှတ်ရန်
   - `.add_edge(from, to, condition=...)` ကို သုံးပြီး အခြေအနေတစ်ခုအရ များပြီးစီးမှုလမ်းကြောင်း ချိတ်ဆက်ရန်
   - `.build()` ကို ခေါ်သုံး၍ workflow ကို အဆုံးသတ်ရန်

2. **အခြေအနေအရ လမ်းကြောင်းရွေးချယ်ခြင်း**
   - အခြေအနေ function များသည် `AgentExecutorResponse` ကို စစ်ဆေးသည်
   - ဖွဲ့စည်းပြီးဖြစ်သော output များကို ဖြုတ်ထုတ်ခြင်းဖြင့် လမ်းကြောင်း ဆုံးဖြတ်ချက်ပြုလုပ်သည်
   - လမ်းကြောင်းကို အလှုအယက်ဖြစ်စေရန်အတွက် `True` ပြန်အပ်ရ며၊ မသုံးလိုပါက `False` ပြန်အပ်သည်

3. **ကိရိယာများ ပေါင်းစပ်ခြင်း**
   - Python functions များကို AI ကိရိယာများအဖြစ် ပြောင်းလဲရန် `@ai_function` ကိုသုံးသည်
   - Agents များက ကိရိယာများကို လိုအပ်သလို အလိုအလျောက် ခေါ်ယူသည်
   - ကိရိယာများက JSON ကို ပြန်အပ်၍ Agents များစီဖြတ်နိုင်သည်

4. **ဖွဲ့စည်းသွားသော Output များ**
   - Pydantic မော်ဒယ်များကို အသုံးပြုပြီး data ကို အမျိုးအစားလုံခြုံစွာ ထုတ်ယူသည်
   - Agents ဖန်တီးရာတွင် `response_format=MyModel` ကို သတ်မှတ်ရန်
   - ပြန်လာသော response များကို `Model.model_validate_json()` ဖြင့် စစ်ဆေးရန်

5. **စိတ်ကြိုက် Executor များ**
   - Workflow ပိုင်းများ ဖန်တီးရန် `@executor(id="...")` ကို သုံးခြင်း
   - Executors များသည် data ပြောင်းလဲခြင်း သို့မဟုတ် အသေးစား ဖျက်ဆီးမှုများ ပြုလုပ်နိုင်သည်
   - `ctx.yield_output()` ဖြင့် workflow ရလဒ်များ ထုတ်ပေးသည်

### 🚀 အပြင်အဆင်မှာ အသုံးပြုနိုင်မှုများ

- **ခရီးစဉ်စာရင်းရေးခြင်း**: ရရှိနိုင်မှု စစ်ဆေးခြင်း၊ အစားထိုးများ အကြံပြုခြင်း၊ ရွေးချယ်စရာများနှိုင်းယှဉ်ခြင်း
- **ဖောက်သည်ဆက်ဆံရေး**: ပြဿနာအမျိုးအစား၊ စိတ်ထားများ၊ ဦးစားပေးမှုအရ လမ်းကြောင်း ချိတ်ဆက်ခြင်း
- **အွန်လိုင်းရောင်းဝယ်ရေး**: စတော့ရှယ်ယာ စစ်ဆေးခြင်း၊ အစားထိုးများ အကြံပြုခြင်း၊ မှာယူမှုများ ကိုင်တွယ်ခြင်း
- **အကြောင်းအရာ စီမံခန့်ခွဲမှု**: လိမ်လည်မှုမှတ်တမ်း၊ အသုံးပြုသူ အမူအရာအရ လမ်းကြောင်း ချိတ်ဆက်ခြင်း
- **အတည်ပြုခြင်း လုပ်ငန်းစဉ်များ**: ပမာဏ၊ အသုံးပြုသူ အခန်းကဏ္ဍ၊ အန္တရာယ်အဆင့်အရ လမ်းကြောင်း ချိတ်ဆက်ခြင်း
- **အဆင့်စုံ သုံးသပ်ခြင်း**: ဒေတာအရည်အသွေး၊ ပြည့်စုံမှုအရ လမ်းကြောင်း ချိတ်ဆက်ခြင်း

### 📚 နောက်တစ်ဆင့်များ

- အခြေအနေ ပိုမိုရှုပ်ထွေးသော ( criteria များ တစ်ခါထဲ သတ်မှတ်ခြင်း) ထည့်သွင်းရန်
- Workflow အခြေအနေ စီမံခန့်ခွဲမှုဖြင့် loop များ တည်ဆောက်ရန်
- pun-workflow များ ထည့်သွင်း၍ ပြန်လည်အသုံးချနိုင်သော component များ ဖန်တီးရန်
- အမှန်တကယ် API များနှင့်ပေါင်းစပ်ခြင်း (ဟိုတယ်စာရင်း, စတော့ရှယ်ယာစနစ်များ)
- အမှား ကိုင်တွယ်ခြင်း နှင့် မရောက်နိုင်သော လမ်းကြောင်းများ ထည့်သွင်းရန်
- ပြုလုပ်ထားသော visualization ကိရိယာများဖြင့် workflow များကို မြင်ကွင်းမြင်ရေးရန်


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**အကြောင်းကြားချက်**  
ဤစာတမ်းကို AI ဘာသာပြန်စနစ်ဖြစ်သည့် [Co-op Translator](https://github.com/Azure/co-op-translator) မှတဆင့် ဘာသာပြန်ထားသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားသော်လည်း ပြန်လည်ဘာသာပြန်ခြင်းသည် အမှားများ သို့မဟုတ် မှားမှုများ ပါဝင်နိုင်ကြောင်း ကျေးဇူးပြု၍ သတိပြုပါ။ မူရင်းစာတမ်းကို မူရင်းဘာသာဖြင့်သာ တရားဝင်အရင်းအမြစ်အဖြစ်ယူဆသင့်ပါသည်။ အရေးပါတဲ့သတင်းအချက်အလက်များအတွက် လူကြီးမင်းတို့အား ပညာရှင်လူသားများမှ ဘာသာပြန်ခြင်းကို အကြံပြုပါသည်။ ဤဘာသာပြန်ချက်ကိုအသုံးပြုမှုမှ ဖြစ်ပေါ်လာသော နားလည်မှုမှားယွင်းမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မထမ်းဆောင်ပါ။
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
